Author: Omar El Malki (omar.elmalki@epfl.ch)

### Semantic Role Labling on ROCStories

In [20]:
from typing import List
import re
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

from allennlp.common import JsonDict
# Semantic Role Labeling with BERT : https://github.com/Riccorl/transformer-srl
from transformer_srl import dataset_readers, models, predictors

In [21]:
def sentence_to_srl(sentence: str) -> JsonDict:
    """
    Extracts Semantic Roles from a sentence.

    :param sentence: sentence from which to extract semantic roles labels.
    :return: semantic_roles as PropBank English SRLs.
    """

    # Pre-trained model with BERT fine-tuned to predict PropBank SRLs on CoNLL 2012 dataset.
    predictor = predictors.SrlTransformersPredictor.from_path(
        "../data/pre-trained-transformer-srl/srl_bert_base_conll2012.tar.gz", "transformer_srl")

    # More documentation: https://docs.allennlp.org/models/main/models/structured_prediction/predictors/srl/
    semantic_roles: JsonDict = predictor.predict(sentence)
    return semantic_roles

In [3]:
s1 = 'Jenny lived in Florida.'
srl1=sentence_to_srl(s1)
print(f"s1=\'{s1}\', SRL={srl1}\n")

s2 = 'David noticed he had put on a lot of weight recently.'
srl2=sentence_to_srl(s2)
print(f"s2=\'{s2}\', SRL={sentence_to_srl(s2)}\n")

s1='Jenny lived in Florida.', SRL={'verbs': [{'verb': 'lived', 'description': '[ARG0: Jenny] [live.01: lived] [ARGM-LOC: in Florida] .', 'tags': ['B-ARG0', 'B-V', 'B-ARGM-LOC', 'I-ARGM-LOC', 'O'], 'frame': 'live.01', 'frame_score': 0.9999998807907104, 'lemma': 'live'}], 'words': ['Jenny', 'lived', 'in', 'Florida', '.']}

s2='David noticed he had put on a lot of weight recently.', SRL={'verbs': [{'verb': 'noticed', 'description': '[ARG0: David] [notice.01: noticed] [ARG1: he had put on a lot of weight recently] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O'], 'frame': 'notice.01', 'frame_score': 0.9999997615814209, 'lemma': 'notice'}, {'verb': 'had', 'description': 'David noticed he [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}, {'verb': 'put', 'description': 'David noticed [ARG0

In [5]:
def srl_to_triple(srl: JsonDict):
    """
    Extract Knowledge triples from semantic roles

    :param srl: PropBank English SRLs
    :return: knowledge triples as a List of Lists
    """
    res = []
    verbs = srl['verbs']
    n_empty = 0
    for d in verbs:
        tags = d['tags']
        triple = d['description']
        verb = d['verb']
        if 'B-ARG0' in tags:
            arg0 = re.search('\\[ARG0: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[1234M]+.*?: (.*?)\\]', triple)
            for arg1 in args:  
                res.append([arg0,verb,arg1])
        elif 'B-ARG1' in tags:
            arg1 = re.search('\\[ARG1: (.*?)\\]', triple).group(1)
            args = re.findall('\\[ARG[234M]+.*?: (.*?)\\]', triple)
            for arg2 in args:  
                res.append([arg1,verb,arg2])
        else:
            n_empty += 1
            print(f"Verb with no arguments: \n{d}\n")
    if not res:
        print(f"Empty triples for SRL: \n{srl}\n")
    return res

In [6]:
srl_to_triple(srl2)

Verb with no arguments: 
{'verb': 'had', 'description': 'David noticed he [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}



[['David', 'noticed', 'he had put on a lot of weight recently'],
 ['he', 'put', 'a lot of weight'],
 ['he', 'put', 'recently']]

In [7]:
srl_to_triple(srl1)

[['Jenny', 'lived', 'in Florida']]

In [8]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = "../data/rocstories-2017/ROCStories_winter2017.csv"
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep=',', header=0)

In [9]:
srl_df = roc_stories_df.head(50)

In [10]:
# Add Knowledge triples to Dataframe for each sentence in the dataset
for n in range(1,6):
    srl_df[f'srl_s{n}'] = srl_df[f'sentence{n}'].progress_apply(lambda s: srl_to_triple(sentence_to_srl(s)))

Verb with no arguments: 
{'verb': 'had', 'description': 'David noticed he [have.01: had] put on a lot of weight recently .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'have.01', 'frame_score': 1.0, 'lemma': 'have'}

Verb with no arguments: 
{'verb': 'was', 'description': 'Andy [be.03: was] invited to a Halloween party .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.9999990463256836, 'lemma': 'be'}

Verb with no arguments: 
{'verb': 'was', 'description': 'Luke [be.03: was] playing hockey at school .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.9999995231628418, 'lemma': 'be'}

Verb with no arguments: 
{'verb': 'was', 'description': 'Robbie [be.03: was] competing in a cross country meet .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.9999992847442627, 'lemma': 'be'}

Verb with no arguments: 
{'verb': "'d", 'description': 

/opt/anaconda3/envs/storygen/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Verb with no arguments: 
{'verb': 'would', 'description': 'Bill thought a truck [go.04: would] be better for what he needed .', 'tags': ['O', 'O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'go.04', 'frame_score': 0.5918087363243103, 'lemma': 'would'}

Verb with no arguments: 
{'verb': 'was', 'description': "His girlfriend [be.03: was] working and would n't take him .", 'tags': ['O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.999998927116394, 'lemma': 'be'}

Verb with no arguments: 
{'verb': 'was', 'description': "It [be.03: was] going to be in the town 's July 4th parade .", 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.9999990463256836, 'lemma': 'be'}

Verb with no arguments: 
{'verb': 'going', 'description': "It was [go.04: going] to be in the town 's July 4th parade .", 'tags': ['O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'go.

Verb with no arguments: 
{'verb': 'did', 'description': 'She [do.01: did] not receive another letter back in the mail .', 'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'do.01', 'frame_score': 0.9999996423721313, 'lemma': 'do'}

Verb with no arguments: 
{'verb': 'will', 'description': 'I had a nice time and [go.04: will] visit my friend this year .', 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O'], 'frame': 'go.04', 'frame_score': 0.7642723917961121, 'lemma': 'will'}

Verb with no arguments: 
{'verb': 'would', 'description': 'He realized he [go.04: would] need a new Xbox .', 'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O'], 'frame': 'go.04', 'frame_score': 0.7586804032325745, 'lemma': 'would'}

Verb with no arguments: 
{'verb': 'is', 'description': 'The movement [be.03: is] gaining momentum !', 'tags': ['O', 'O', 'B-V', 'O', 'O', 'O'], 'frame': 'be.03', 'frame_score': 0.9999990463256836, 'lemma': 'be'}

Verb with no argume

In [18]:
srl_df.head(5)

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,srl_s1,srl_s2,srl_s3,srl_s4,srl_s5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better.","[[David, noticed, he had put on a lot of weight recently], [he, put, a lot of weight], [he, put, recently]]","[[He, examined, his habits], [He, examined, to try and figure out the reason], [He, try, the reason], [He, figure, the reason]]","[[He, realized, he 'd been eating too much fast food lately], [he, eating, too much fast food], [he, eating, lately]]","[[He, stopped, going to burger places], [He, going, to burger places], [He, started, a vegetarian diet]]","[[to feel much better, started, After a few weeks], [he, feel, much better]]"
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.,"[[Tom, had, a very short temper]]","[[a guest, made, One day], [a guest, made, him very angry]]","[[He, punched, a hole in the wall of his house]]","[[Tom 's guest, became, afraid], [Tom 's guest, left, quickly]]","[[Tom, sat, on his couch], [Tom, sat, filled with regret about his actions], [Tom, filled, with regret about his actions]]"
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for the event.,"[[Marcus, needed, clothing], [Marcus, needed, for a business casual event]]","[[All of his clothes, were, either too formal or too casual]]","[[He, decided, to buy a pair of khakis], [He, buy, a pair of khakis]]","[[he, bought, The pair], [The pair he bought, fit, him], [The pair he bought, fit, perfectly]]","[[Marcus, was, happy to have the right clothes for the event], [Marcus, have, the right clothes for the event]]"
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and haul it with his car.,Bill thought a truck would be better for what he needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in his ways with conventional thinking.,He ended up buying the truck he wanted despite Bobby's advice.,"[[Bobby, thought, Bill should buy a trailer and haul it with his car], [Bill, should, buy a trailer and haul], [Bill, buy, should], [Bill, buy, a trailer], [Bill, haul, should], [Bill, haul, it], [Bill, haul, with his car]]","[[Bill, thought, a truck would be better for what he needed], [a truck, be, would], [a truck, be, better for what he needed], [he, needed, what]]","[[Bobby, pointed, two vehicles were much more expensive], [two vehicles, were, much more expensive]]","[[Bill, set, in his ways], [Bill, set, with conventional thinking]]","[[He, ended, buying the truck he wanted], [He, ended, despite Bobby 's advice], [He, buying, the truck he wanted], [He, buying, despite Bobby 's advice], [he, wanted, the truck]]"
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in advance but to no avail.,He decided to learn to sing to overcome his handicap.,He then made all his sermons into music and sang them on Sundays.,His congregation was delighted and so was he.,"[[John, was, a pastor with a very bad memory]]","[[He, tried, to memorize his sermons many days in advance], [He, tried, but to no avail], [He, memorize, his sermons], [He, memorize, many days in advance]]","[[He, decided, to learn to sing to overco

Examples of sentences with no knowledge triples.

In [12]:
# No verb detected
sentence_to_srl("That evening, the phone rang.")

{'verbs': [], 'words': ['That', 'evening', ',', 'the', 'phone', 'rang', '.']}

In [13]:
# No verb detected
sentence_to_srl("The female voice above her announced final boarding to Soren's flight.")

{'verbs': [],
 'words': ['The',
  'female',
  'voice',
  'above',
  'her',
  'announced',
  'final',
  'boarding',
  'to',
  'Soren',
  "'s",
  'flight',
  '.']}

In [16]:
# No verb detected
sentence_to_srl("Andy was disappointed with his new, bold, green hair color.")

{'verbs': [{'verb': 'was',
   'description': 'Andy [be.01: was] disappointed with his new , bold , green hair color .',
   'tags': ['O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
   'frame': 'be.01',
   'frame_score': 0.8844010829925537,
   'lemma': 'be'}],
 'words': ['Andy',
  'was',
  'disappointed',
  'with',
  'his',
  'new',
  ',',
  'bold',
  ',',
  'green',
  'hair',
  'color',
  '.']}

In [14]:
# Only 1 argument
sentence_to_srl("Everybody cheered!")

{'verbs': [{'verb': 'cheered',
   'description': '[ARG0: Everybody] [cheer.02: cheered] !',
   'tags': ['B-ARG0', 'B-V', 'O'],
   'frame': 'cheer.02',
   'frame_score': 0.999954342842102,
   'lemma': 'cheer'}],
 'words': ['Everybody', 'cheered', '!']}

In [15]:
# Only 1 argument
sentence_to_srl("Eric's wife passed away.")

{'verbs': [{'verb': 'passed',
   'description': "[ARG1: Eric 's wife] [pass.16: passed] away .",
   'tags': ['B-ARG1', 'I-ARG1', 'I-ARG1', 'B-V', 'O', 'O'],
   'frame': 'pass.16',
   'frame_score': 0.9999988079071045,
   'lemma': 'pass'}],
 'words': ['Eric', "'s", 'wife', 'passed', 'away', '.']}